### Week6: Digital Humanities - Vector Space Model


In [25]:
import os
import lxml.etree
import tarfile
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import nltk
import nltk.tokenize
import math
import random


tf = tarfile.open('theatre-classique.tar.gz','r')
tf.extractall('data')

In [26]:
subgenres = ('Comédie', 'Tragédie', 'Tragi-comédie')
#print(subgenres)
plays, titles, genres = [], [], []
authors, years = [],[]

In [27]:
for fn in os.scandir('data/theatre-classique'):
    # Only include XML files
    if not fn.name.endswith('.xml'):
        continue
    tree   = lxml.etree.parse(fn.path)
    genre  = tree.find('//genre')
    title  = tree.find('//title')
    author = tree.find('//author')
    year   = tree.find('//date')
    if genre is not None and genre.text in subgenres:
        lines = []
        for line in tree.xpath('//l|//p'):
            lines.append(' '.join(line.itertext()))
        text = '\n'.join(lines)
        plays.append(text)
        genres.append(genre.text)
        titles.append(title.text)
        authors.append(author.text)
        if year is not None:
            years.append(year.text)

plays = np.array(plays)
genres = np.array(genres)
titles = np.array(titles)
authors = np.array(authors)
years = np.array(years)

In [28]:
print(len(plays), len(genres), len(titles), len(authors), len(years))


498 498 498 498 208


In [29]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Veda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def is_punct(string):
    """Check if STRING is a punctuation marker or a sequence of
       punctuation markers.
    """
    return PUNCT_RE.match(string) is not None

PUNCT_RE = re.compile(r'[^\w\s]+$')

In [31]:
def preprocess_text(text, language='French', lowercase=True):
    if lowercase:
        text = text.lower()
    if (language == 'French'):
        text = re.sub("-", " ", text)
        text = re.sub("l'", "le ", text)
        text = re.sub("d'", "de ", text)
        text = re.sub("c'", "ce ", text)
        text = re.sub("j'", "je ", text)
        text = re.sub("m'", "me ", text)
        text = re.sub("qu'", "que ", text)
        text = re.sub("'", " ' ", text)
        text = re.sub("quelqu'", "quelque ", text)
        text = re.sub("aujourd'hui", "aujourdhui", text)
    tokens = nltk.tokenize.word_tokenize(text, language=language)
    tokens = [token for token in tokens if not is_punct(token)]
    return tokens

In [32]:
plays_tok = [preprocess_text(play, 'French') for play in plays]


In [33]:
def extract_vocabulary(tokenized_corpus, min_count=1, max_count=float('inf')):
    vocabulary = collections.Counter()
    for document in tokenized_corpus:
        vocabulary.update(document)
    vocabulary = {word for word, count in vocabulary.items()
                  if count >= min_count and count <= max_count}
    return sorted(vocabulary)

vocabulary = extract_vocabulary(plays_tok, min_count=2)
print("Length of vocabulary: ",len(vocabulary))


Length of vocabulary:  38410


#### 1. Represent each play by a vector with only the tf component. You can apply some preprocessing before generating this vector representation.

In [44]:
def corpus2dtm(tokenized_corpus, vocabulary):
    document_term_matrix = []
    for document in tokenized_corpus:
        document_counts = collections.Counter(document)
        row = [document_counts[word] for word in vocabulary]
        document_term_matrix.append(row)
    return np.array(document_term_matrix)

In [45]:
document_term_matrix = np.array(corpus2dtm(plays_tok, vocabulary))
print(f"document-term matrix with "
      f"|D| = {document_term_matrix.shape[0]} documents and "
      f"|V| = {document_term_matrix.shape[1]} words.")

document-term matrix with |D| = 498 documents and |V| = 38410 words.


In [46]:
print("Converted doc into vectors :\n",document_term_matrix)
print("\nLength of matrix: \n", len(document_term_matrix))
print("\nSize of matrix: \n", document_term_matrix.shape)

Converted doc into vectors :
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Length of matrix: 
 498

Size of matrix: 
 (498, 38410)


#### write a Python function to return a list of terms present in more than 10 documents.

In [65]:
#count = 0
terms = []
voc = []
#document_term_matrix = int(document_term_matrix)
    #print(all_doc[40])
#     if(all_doc[index] > 1):
#         terms = document_term_matrix[index]
#         count += 1
for all_doc in range(0,38410):
    if len(np.where(document_term_matrix[:,all_doc]>0)[0])>10:
        terms.append(all_doc)
        voc.append(vocabulary[all_doc])

for i in range(len(terms)):
    print(terms[i] ,':', voc[i])

3 : 1
5 : 10
9 : 11
12 : 12
14 : 13
16 : 14
18 : 15
22 : 16
114 : 17
141 : 1723.
143 : 1725
213 : 18
219 : 19
223 : 1er
225 : 2
227 : 20
230 : 2006
232 : 2008
234 : 2009
236 : 2010
238 : 2011
240 : 2012
241 : 2012.
242 : 2013
244 : 2014
247 : 21
249 : 22
256 : 26
258 : 27
260 : 28
266 : 3
267 : 3.
278 : 4
285 : 5
291 : 6
297 : 7
298 : 7.
302 : 8
309 : 9
312 : a
313 : a.
322 : abaissant
323 : abaisse
324 : abaissement
327 : abaisser
333 : abaissé
337 : abandon
338 : abandonna
339 : abandonnai
343 : abandonnant
344 : abandonne
346 : abandonnent
347 : abandonner
353 : abandonnerez
356 : abandonnez
359 : abandonnons
360 : abandonné
361 : abandonnée
363 : abandonnés
364 : abat
372 : abattre
373 : abattu
374 : abattue
376 : abattus
378 : abbé
379 : abbés
395 : abhorre
397 : abhorrer
399 : abhorré
403 : abject
407 : abjure
408 : abjurer
409 : abjurez
413 : abois
416 : abolir
420 : abominable
423 : abondamment
424 : abondance
428 : abonde
431 : abord
435 : abordant
436 : aborde
438 : aborder
4

3148 : assurée
3150 : assurément
3151 : assurés
3164 : astre
3166 : astres
3169 : astrologue
3181 : athalie
3185 : athènes
3190 : athéniens
3198 : atrabilaire
3201 : atroce
3203 : atrocité
3207 : attacha
3208 : attachaient
3210 : attachait
3211 : attachant
3212 : attache
3213 : attachement
3215 : attachent
3216 : attacher
3221 : attachez
3223 : attaché
3224 : attachée
3226 : attachés
3228 : attaqua
3233 : attaquant
3234 : attaque
3235 : attaquent
3236 : attaquer
3243 : attaquez
3247 : attaqué
3248 : attaquée
3254 : atteindre
3256 : atteint
3257 : atteinte
3258 : atteintes
3259 : atteints
3266 : attend
3267 : attendaient
3268 : attendais
3269 : attendait
3270 : attendant
3271 : attende
3272 : attendent
3274 : attendez
3275 : attendiez
3276 : attendions
3280 : attendons
3282 : attendra
3283 : attendrai
3284 : attendrais
3285 : attendrait
3287 : attendre
3288 : attendrez
3289 : attendri
3290 : attendrie
3292 : attendrir
3294 : attendris
3300 : attendrissent
3301 : attendrissez
3303 : atte

6095 : charitable
6098 : charité
6100 : charivari
6102 : charlatan
6105 : charlatans
6108 : charles
6114 : charma
6117 : charmait
6119 : charmant
6120 : charmante
6121 : charmantes
6122 : charmants
6123 : charme
6124 : charment
6125 : charmer
6131 : charmes
6134 : charmez
6137 : charmé
6138 : charmée
6140 : charmés
6153 : charte
6154 : chartre
6156 : chassa
6160 : chassant
6161 : chasse
6163 : chasser
6165 : chasserai
6169 : chasseur
6170 : chasseurs
6171 : chassez
6174 : chassons
6177 : chassé
6178 : chassée
6179 : chassés
6180 : chaste
6183 : chastes
6185 : chat
6197 : chats
6199 : chaud
6200 : chaude
6215 : chausses
6222 : chauve
6224 : chaîne
6225 : chaînes
6226 : che
6227 : chef
6228 : chefs
6230 : chemin
6233 : chemins
6234 : cheminée
6235 : chemise
6240 : cher
6243 : cherchaient
6244 : cherchais
6245 : cherchait
6246 : cherchant
6247 : cherche
6248 : cherchent
6249 : chercher
6250 : cherchera
6251 : chercherai
6252 : chercherais
6259 : cherches
6263 : cherchez
6264 : cherchiez
6

8566 : craignez
8567 : craigniez
8569 : craignis
8571 : craignit
8572 : craignons
8573 : craindra
8574 : craindrai
8576 : craindrais
8577 : craindrait
8578 : craindre
8579 : craindrez
8583 : crains
8584 : craint
8585 : crainte
8586 : craintes
8587 : craintif
8589 : craintive
8609 : crayon
8622 : creuse
8623 : creuser
8626 : creusé
8628 : creux
8632 : crever
8634 : crevé
8636 : cri
8638 : criait
8639 : criant
8643 : crie
8645 : crier
8652 : criez
8653 : crime
8654 : crimes
8655 : criminel
8656 : criminelle
8657 : criminelles
8658 : criminels
8663 : cris
8665 : crise
8667 : crispin
8670 : cristal
8674 : critique
8676 : critiquer
8677 : critiques
8683 : crié
8686 : croc
8695 : croie
8696 : croient
8697 : croira
8698 : croirai
8699 : croiraient
8700 : croirais
8701 : croirait
8702 : croiras
8703 : croire
8704 : croirez
8705 : croiriez
8708 : croiront
8709 : crois
8717 : croissant
8721 : croissent
8724 : croisés
8725 : croit
8726 : croix
8741 : croupe
8746 : croyable
8748 : croyaient
8749 :

11214 : dédit
11219 : dédommage
11223 : dédommager
11229 : déesse
11230 : déesses
11236 : défaire
11238 : défait
11239 : défaite
11240 : défaites
11241 : défaits
11243 : défaut
11244 : défauts
11247 : défectueux
11248 : défend
11251 : défendait
11252 : défendant
11253 : défende
11254 : défendent
11256 : défendez
11258 : défendit
11259 : défendons
11260 : défendra
11261 : défendrai
11266 : défendre
11271 : défends
11272 : défendu
11273 : défendue
11275 : défendus
11277 : défense
11278 : défenses
11279 : défenseur
11280 : défenseurs
11289 : défi
11293 : défiance
11297 : défiant
11300 : défie
11302 : défier
11306 : défiez
11312 : défiguré
11313 : défigurée
11317 : définir
11326 : défié
11333 : défunt
11334 : défunte
11336 : défère
11337 : déférence
11339 : déférer
11342 : dégage
11346 : dégager
11349 : dégagé
11350 : dégagée
11367 : dégoût
11372 : dégoûte
11373 : dégoûter
11374 : dégoûts
11375 : dégoûté
11376 : dégoûtée
11382 : dégrader
11391 : déguerpir
11396 : déguisant
11397 : déguise


...

27927 : pâlir
27928 : pâlis
27934 : pâlissez
27935 : pâlit
27936 : pâme
27943 : pâris
27944 : pâte
27952 : pâture
27955 : pèche
27962 : père
27963 : pères
27964 : pèse
27968 : pécher
27972 : péché
27973 : péchés
27974 : pécore
27977 : pédant
27981 : pédants
27982 : pégase
27990 : pénible
27992 : pénibles
27993 : pénitence
27996 : pénètre
27997 : pénètrent
28003 : pénétrant
28004 : pénétrante
28006 : pénétrants
28007 : pénétration
28008 : pénétrer
28009 : pénétrez
28011 : pénétrons
28012 : pénétré
28013 : pénétrée
28014 : pénétrés
28020 : péri
28026 : péril
28028 : périlleux
28029 : périls
28036 : périr
28037 : périra
28038 : périrai
28040 : périrait
28041 : périras
28043 : périrez
28044 : périrons
28045 : périront
28046 : péris
28053 : périsse
28054 : périssent
28056 : périssez
28059 : périssons
28060 : périt
28063 : pérou
28068 : pétille
28072 : pétri
28080 : pétulance
28085 : pêche
28086 : pêcher
28088 : pêché
28090 : pêle
28094 : pû
28095 : pût
28097 : q
28103 : qu
28118 : qualité
2

29586 : remédier
29593 : renaissant
29598 : renaissent
29599 : renard
29604 : renaît
29606 : renaître
29612 : rencontra
29613 : rencontrai
29617 : rencontrant
29618 : rencontre
29619 : rencontrent
29620 : rencontrer
29632 : rencontré
29633 : rencontrée
29634 : rencontrés
29635 : rend
29636 : rendaient
29637 : rendais
29638 : rendait
29639 : rendant
29640 : rende
29641 : rendent
29643 : rendez
29644 : rendiez
29646 : rendirent
29647 : rendis
29649 : rendit
29650 : rendons
29652 : rendra
29653 : rendrai
29654 : rendraient
29655 : rendrais
29656 : rendrait
29657 : rendras
29658 : rendre
29659 : rendrez
29661 : rendrons
29662 : rendront
29663 : rends
29664 : rendu
29665 : rendue
29667 : rendus
29668 : rendît
29673 : renferme
29675 : renfermer
29677 : renfermez
29679 : renfermé
29680 : renfermée
29700 : renom
29701 : renomme
29703 : renommé
29704 : renommée
29707 : renonce
29708 : renoncer
29712 : renoncez
29715 : renoncé
29716 : renonçant
29721 : renouer
29725 : renouveler
29727 : renouvel

31106 : régné
31107 : régularité
31108 : régulier
31110 : régulière
31111 : régulièrement
31120 : réjoui
31121 : réjouie
31122 : réjouir
31126 : réjouis
31130 : réjouissance
31138 : réjouissez
31140 : réjouit
31149 : répand
31151 : répandait
31152 : répandant
31153 : répande
31154 : répandent
31155 : répandez
31158 : répandit
31164 : répandre
31167 : répands
31168 : répandu
31169 : répandue
31171 : répandus
31175 : réparation
31177 : répare
31179 : réparer
31186 : réparez
31194 : réparé
31201 : réplique
31202 : répliquer
31203 : répliques
31204 : répliquez
31207 : répond
31210 : répondait
31211 : répondant
31212 : réponde
31213 : répondent
31215 : répondez
31218 : répondis
31220 : répondit
31222 : répondra
31223 : répondrai
31225 : répondrais
31227 : répondras
31228 : répondre
31229 : répondrez
31233 : réponds
31234 : répondu
31236 : répons
31237 : réponse
31238 : réponses
31245 : réprimer
31246 : réprimez
31262 : république
31268 : répugnance
31270 : répugne
31274 : réputation
31279 :

33687 : susceptible
33691 : suscite
33700 : suspect
33701 : suspecte
33703 : suspects
33704 : suspend
33710 : suspendez
33712 : suspendons
33716 : suspendre
33718 : suspends
33719 : suspendu
33720 : suspendue
33721 : suspendus
33722 : suspens
33730 : sut
33745 : sylla
33746 : syllabe
33757 : sympathie
33763 : symphonie
33770 : syndic
33779 : syrie
33784 : système
33797 : sagit
33809 : sappelle
33816 : sapprête
33850 : sen
33858 : sest
33871 : sil
33872 : sils
33900 : sy
33920 : sur
33921 : surs
33937 : sèche
33940 : sème
33945 : séance
33951 : sécher
33957 : sécurité
33960 : séditieux
33961 : sédition
33963 : séducteur
33964 : séducteurs
33966 : séductions
33969 : séduire
33972 : séduisant
33979 : séduit
33980 : séduite
33981 : séduits
33987 : séjour
33994 : sémiramis
33996 : sénat
33997 : sénateur
33998 : sénateurs
34000 : sénèque
34003 : sénéchaux
34007 : séparant
34008 : séparation
34010 : sépare
34011 : séparent
34012 : séparer
34016 : séparez
34017 : séparons
34020 : sép

36279 : venue
36280 : venues
36281 : venus
36285 : ver
36288 : verbal
36292 : verbiage
36295 : verdure
36305 : vermeil
36306 : vermeille
36310 : vernis
36312 : verra
36313 : verrai
36314 : verraient
36315 : verrais
36316 : verrait
36317 : verras
36318 : verre
36319 : verres
36320 : verrez
36321 : verriez
36322 : verrions
36324 : verrons
36325 : verront
36327 : verrous
36330 : vers
36331 : versa
36334 : versailles
36336 : versait
36337 : versant
36338 : verse
36339 : versent
36340 : verser
36349 : versez
36351 : versification
36356 : versé
36359 : versés
36360 : vert
36366 : vertigo
36368 : verts
36369 : vertu
36373 : vertueuse
36375 : vertueuses
36376 : vertueux
36377 : vertus
36378 : verve
36383 : veste
36389 : veuf
36391 : veuille
36392 : veuillent
36394 : veuillez
36395 : veulent
36396 : veut
36397 : veuvage
36398 : veuve
36399 : veuves
36401 : veux
36404 : vi
36411 : viande
36414 : vians
36415 : viant
36417 : vice
36419 : vices
36422 : vicieux
36425 : vicomte
36428 : victime
36429 

#### 2. For each genre, it is possible to generate a “profile”, in the form of a single vector representing the entire set of plays corresponding to this genre. Build such a profile for each of the three genres (Comedy, Tragedy and Tragicomedy).

In [13]:
tr_means = document_term_matrix[np.array(genres) == 'Tragédie'].mean(axis=0)
co_means = document_term_matrix[genres == 'Comédie'].mean(axis=0)
tc_means = document_term_matrix[genres == 'Tragi-comédie'].mean(axis=0)

#### 3. How many terms with a weight strictly larger than 0 do you have in each text genre profile?

In [14]:
for genre_ in subgenres:
    mins = document_term_matrix[np.array(genres) == genre_].mean(axis=0)
    nonzero_terms = np.sum(mins > 0)
    print(f"Number of non-zero terms in {genre_} is {nonzero_terms}")


Number of non-zero terms in Comédie is 33990
Number of non-zero terms in Tragédie is 25642
Number of non-zero terms in Tragi-comédie is 16401


#### 4. Select randomly 10 plays for each text genre. Represent each play by a vector.

In [23]:
for genre_ in subgenres:
    genre_plays = document_term_matrix[np.array(genres) == genre_]
    genre_plays = genre_plays[np.random.randint(0, len(genre_plays), 10)]
    print(f"Each play in {genre_} is a dimension of {len(genre_plays[0])} with size {genre_plays.shape}")


Each play in Comédie is a dimension of 38410 with size (10, 38410)
Each play in Tragédie is a dimension of 38410 with size (10, 38410)
Each play in Tragi-comédie is a dimension of 38410 with size (10, 38410)


#### 5. For each text genre and play, how many terms with a weight strictly larger than 0 do you have in the vector?

In [24]:
for genre_ in subgenres:
    genre_plays = document_term_matrix[np.array(genres) == genre_]
    for index, play_ in enumerate(genre_plays):
        nonzero_terms = np.sum(play_ > 0)
        print(f"Number of terms > 0 in play {index}/{len(genre_plays)} of {genre_} is {nonzero_terms}")
    
    print("\n")


Number of terms > 0 in play 0/310 of Comédie is 1262
Number of terms > 0 in play 1/310 of Comédie is 1608
Number of terms > 0 in play 2/310 of Comédie is 610
Number of terms > 0 in play 3/310 of Comédie is 2675
Number of terms > 0 in play 4/310 of Comédie is 1305
Number of terms > 0 in play 5/310 of Comédie is 1148
Number of terms > 0 in play 6/310 of Comédie is 1394
Number of terms > 0 in play 7/310 of Comédie is 1346
Number of terms > 0 in play 8/310 of Comédie is 2670
Number of terms > 0 in play 9/310 of Comédie is 1245
Number of terms > 0 in play 10/310 of Comédie is 1512
Number of terms > 0 in play 11/310 of Comédie is 1622
Number of terms > 0 in play 12/310 of Comédie is 938
Number of terms > 0 in play 13/310 of Comédie is 943
Number of terms > 0 in play 14/310 of Comédie is 1205
Number of terms > 0 in play 15/310 of Comédie is 1350
Number of terms > 0 in play 16/310 of Comédie is 790
Number of terms > 0 in play 17/310 of Comédie is 1639
Number of terms > 0 in play 18/310 of Comé

Number of terms > 0 in play 146/150 of Tragédie is 2223
Number of terms > 0 in play 147/150 of Tragédie is 2302
Number of terms > 0 in play 148/150 of Tragédie is 2013
Number of terms > 0 in play 149/150 of Tragédie is 2247


Number of terms > 0 in play 0/38 of Tragi-comédie is 2690
Number of terms > 0 in play 1/38 of Tragi-comédie is 2665
Number of terms > 0 in play 2/38 of Tragi-comédie is 2552
Number of terms > 0 in play 3/38 of Tragi-comédie is 2662
Number of terms > 0 in play 4/38 of Tragi-comédie is 2475
Number of terms > 0 in play 5/38 of Tragi-comédie is 2189
Number of terms > 0 in play 6/38 of Tragi-comédie is 2734
Number of terms > 0 in play 7/38 of Tragi-comédie is 2073
Number of terms > 0 in play 8/38 of Tragi-comédie is 1939
Number of terms > 0 in play 9/38 of Tragi-comédie is 2686
Number of terms > 0 in play 10/38 of Tragi-comédie is 2230
Number of terms > 0 in play 11/38 of Tragi-comédie is 2411
Number of terms > 0 in play 12/38 of Tragi-comédie is 2478
Number of terms >

#### 6. For each text genre and play, how many terms with a weight strictly equal to 1 do you have in the vector?


In [25]:
for genre_ in subgenres:
    genre_plays = document_term_matrix[np.array(genres) == genre_]
    for index, play_ in enumerate(genre_plays):
        nonzero_terms = np.sum(play_ == 1)
        print(f"Number of terms == 1 in play {index}/{len(genre_plays)} of {genre_} is {nonzero_terms}")
    print("\n")


Number of terms == 1 in play 0/310 of Comédie is 758
Number of terms == 1 in play 1/310 of Comédie is 1026
Number of terms == 1 in play 2/310 of Comédie is 479
Number of terms == 1 in play 3/310 of Comédie is 1603
Number of terms == 1 in play 4/310 of Comédie is 809
Number of terms == 1 in play 5/310 of Comédie is 687
Number of terms == 1 in play 6/310 of Comédie is 841
Number of terms == 1 in play 7/310 of Comédie is 798
Number of terms == 1 in play 8/310 of Comédie is 1436
Number of terms == 1 in play 9/310 of Comédie is 731
Number of terms == 1 in play 10/310 of Comédie is 1067
Number of terms == 1 in play 11/310 of Comédie is 969
Number of terms == 1 in play 12/310 of Comédie is 610
Number of terms == 1 in play 13/310 of Comédie is 651
Number of terms == 1 in play 14/310 of Comédie is 684
Number of terms == 1 in play 15/310 of Comédie is 683
Number of terms == 1 in play 16/310 of Comédie is 460
Number of terms == 1 in play 17/310 of Comédie is 1005
Number of terms == 1 in play 18/3

Number of terms == 1 in play 140/150 of Tragédie is 1212
Number of terms == 1 in play 141/150 of Tragédie is 1165
Number of terms == 1 in play 142/150 of Tragédie is 1275
Number of terms == 1 in play 143/150 of Tragédie is 1183
Number of terms == 1 in play 144/150 of Tragédie is 1503
Number of terms == 1 in play 145/150 of Tragédie is 1161
Number of terms == 1 in play 146/150 of Tragédie is 1216
Number of terms == 1 in play 147/150 of Tragédie is 1234
Number of terms == 1 in play 148/150 of Tragédie is 1087
Number of terms == 1 in play 149/150 of Tragédie is 1165


Number of terms == 1 in play 0/38 of Tragi-comédie is 1419
Number of terms == 1 in play 1/38 of Tragi-comédie is 1401
Number of terms == 1 in play 2/38 of Tragi-comédie is 1497
Number of terms == 1 in play 3/38 of Tragi-comédie is 1366
Number of terms == 1 in play 4/38 of Tragi-comédie is 1249
Number of terms == 1 in play 5/38 of Tragi-comédie is 1097
Number of terms == 1 in play 6/38 of Tragi-comédie is 1454
Number of terms